<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#0.-Dataset" data-toc-modified-id="0.-Dataset-1">0. Dataset</a></span></li><li><span><a href="#1.-Preprocessing" data-toc-modified-id="1.-Preprocessing-2">1. Preprocessing</a></span><ul class="toc-item"><li><span><a href="#1.1-Data-Import" data-toc-modified-id="1.1-Data-Import-2.1">1.1 Data Import</a></span></li><li><span><a href="#1.2-Data-Preprocessing" data-toc-modified-id="1.2-Data-Preprocessing-2.2">1.2 Data Preprocessing</a></span><ul class="toc-item"><li><span><a href="#1.2.1-Building-Metadata" data-toc-modified-id="1.2.1-Building-Metadata-2.2.1">1.2.1 Building Metadata</a></span></li><li><span><a href="#1.2.2-weather-data" data-toc-modified-id="1.2.2-weather-data-2.2.2">1.2.2 weather data</a></span></li><li><span><a href="#1.2.3-Train/Test-Dataset" data-toc-modified-id="1.2.3-Train/Test-Dataset-2.2.3">1.2.3 Train/Test Dataset</a></span></li></ul></li></ul></li><li><span><a href="#2.-Baseline-Model" data-toc-modified-id="2.-Baseline-Model-3">2. Baseline Model</a></span><ul class="toc-item"><li><span><a href="#2.-Feature-Engineering" data-toc-modified-id="2.-Feature-Engineering-3.1">2. Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#2.1-ID-features" data-toc-modified-id="2.1-ID-features-3.1.1">2.1 ID features</a></span></li></ul></li></ul></li></ul></div>

https://www.kaggle.com/c/ashrae-energy-prediction/overview

Q: How much does it cost to cool a skyscraper in the summer?
A: A lot! And not just in dollars, but in environmental impact.

Thankfully, significant investments are being made to improve building efficiencies to reduce costs and emissions. The question is, are the improvements working? That’s where you come in. Under pay-for-performance financing, the building owner makes payments based on the difference between their real energy consumption and what they would have used without any retrofits. The latter values have to come from a model. Current methods of estimation are fragmented and do not scale well. Some assume a specific meter type or don’t work with different building types.

In this competition, you’ll develop accurate models of metered building energy usage in the following areas: chilled water, electric, hot water, and steam meters. The data comes from over 1,000 buildings over a three-year timeframe. With better estimates of these energy-saving investments, large scale investors and financial institutions will be more inclined to invest in this area to enable progress in building efficiencies.


## 0. Dataset

**1.train.csv** <br>
>**building_id** - Foreign key for the building metadata.<br>
>**meter** - The meter id code. Read as {0: electricity, 1: chilledwater, 2: steam, 3: hotwater}. Not every building has all meter types. <br>
>**timestamp** - When the measurement was taken <br>
>**meter_reading** - The target variable. Energy consumption in kWh (or equivalent). Note that this is real data with measurement error, which we expect will impose a baseline level of modeling error. **UPDATE: as discussed here, the site 0 electric meter readings are in kBTU.**

**2.building_meta.csv** <br>
>**site_id** - Foreign key for the weather files. <br>
>**building_id** - Foreign key for training.csv <br>
>**primary_use** - Indicator of the primary category of activities for the building based on EnergyStar property type definitions <br>
>**square_feet** - Gross floor area of the building <br>
>**year_built** - Year building was opened <br>
>**loor_count** - Number of floors of the building

**3.weather_(train/test).csv** <br>
Weather data from a meteorological station as close as possible to the site. <br>
>**site_id**<br>
>**air_temperature** - Degrees Celsius <br>
>**cloud_coverage** - Portion of the sky covered in clouds, in oktas <br>
>**dew_temperature** - Degrees Celsius <br>
>**precip_depth_1_hr** - Millimeters <br>
>**sea_level_pressure** - Millibar/hectopascals <br>
>**wind_direction** - Compass direction (0-360) <br>
>**wind_speed** - Meters per second

**4.test.csv**
The submission files use row numbers for ID codes in order to save space on the file uploads. test.csv has no feature data; it exists so you can get your predictions into the correct order.

>**row_id** - Row id for your submission file <br>
>**building_id** - Building id code <br>
>**meter** - The meter id code <br>
>**timestamp** - Timestamps for the test data period <br>

**5.sample_submission.csv**
A valid sample submission.

All floats in the solution file were truncated to **four decimal places**; we recommend you do the same to save space on your file upload. There are gaps in some of the meter readings for both the train and test sets. Gaps in the test set are not revealed or scored.

In [ ]:
import os, sys, datetime, gc

import numpy as np, pandas as pd
from pandas import Series, DataFrame

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline
sns.set()

## 1. Preprocessing

### 1.1 Data Import

In [ ]:
train = pd.read_csv("../input/ashrae-energy-prediction/train.csv")
train.head(10)

In [ ]:
train.info()

In [ ]:
test = pd.read_csv("../input/ashrae-energy-prediction/test.csv")
test.head()

In [ ]:
test.info()

In [ ]:
building_meta = pd.read_csv("../input/ashrae-energy-prediction/building_metadata.csv")
building_meta.head()

In [ ]:
building_meta.info()

In [ ]:
weather_train = pd.read_csv("../input/ashrae-energy-prediction/weather_train.csv")
weather_train.head()

In [ ]:
weather_train.info()

In [ ]:
weather_test = pd.read_csv("../input/ashrae-energy-prediction/weather_test.csv")
weather_test.head()

In [ ]:
weather_test.info()

### 1.2 Data Preprocessing

#### 1.2.1 Building Metadata

In [ ]:
# Building meta data contains null values.

print('Missing Value Percentage:\n',
     building_meta.isnull().sum() / building_meta.shape[0]) 
sns.heatmap(building_meta.isnull(), cbar = False, cmap = 'bwr')

In [ ]:
# shrink the size of DataFrame

def df_col_size(df):
    print('Total Size (Mb): {:.3f}'.format(df.memory_usage().sum() * 1e-6 ))
    print(df.memory_usage() * 1e-6)
    for col in df.columns:
        print(f"{col}: Type: {df[col].dtypes},\
        Min: {df[col].min()},\
        Max: {df[col].max()}")

In [ ]:
df_col_size(building_meta)

In [ ]:
from sklearn.preprocessing import LabelEncoder

use_le = LabelEncoder()

use_le.fit(building_meta.primary_use)
building_meta['primary_use'] = use_le.transform(building_meta.primary_use)

use_le.classes_

In [ ]:
use_le.classes_

In [ ]:
building_meta['site_id'] = building_meta['site_id'].astype(np.int8)
building_meta['building_id'] = building_meta['building_id'].astype(np.int16)
building_meta['square_feet'] = building_meta['square_feet'].astype(np.int32)
building_meta['primary_use'] = building_meta['primary_use'].astype(np.int8)
building_meta['year_built'] = building_meta['year_built'].astype(np.float16)
building_meta['floor_count'] = building_meta['floor_count'].astype(np.float16)

In [ ]:
df_col_size(building_meta)

In [ ]:
# How many unique site ID?

building_meta['site_id'].value_counts().sort_index().plot.bar(figsize = (8,6))

In [ ]:
# What's the missing rate per each site ID
building_meta.groupby('site_id').apply(lambda x: x.isnull().sum() / (x.count() + x.isnull().sum()))

In [ ]:
# Cannot impute missing values only based on site_id. Join the train data and impute all together.

#### 1.2.2 weather data

In [ ]:
# weather is split into train and test data. We combine them together and then analyze.

print('size of train:', weather_train.shape,'\n',
     'size of test', weather_test.shape)

weather_all = pd.concat([weather_train, weather_test], axis = 0, keys = ('train', 'test'))

print('size of all', weather_all.shape)

del weather_train
del weather_test

In [ ]:
#weather data contains null values.

print('Missing Value Percentage:\n',
     weather_all.isnull().sum() / weather_all.shape[0])
plt.figure(figsize = (10,10))
sns.heatmap(weather_all.isnull(), cbar = False, cmap = 'bwr')

In [ ]:
# input data before shrinking the size
weather_all['timestamp'] = pd.to_datetime(weather_all.timestamp)

weather_all['year'] = weather_all['timestamp'].dt.year
weather_all['month'] = weather_all['timestamp'].dt.month
weather_all['day'] = weather_all['timestamp'].dt.day
weather_all['week'] = weather_all['timestamp'].dt.week
weather_all['weekday'] = weather_all['timestamp'].dt.weekday
weather_all['hour'] = weather_all['timestamp'].dt.hour
weather_all['season'] = np.where(weather_all['timestamp'].dt.month <=2, 4, 
         np.where(weather_all['timestamp'].dt.month <= 5, 1,
                 np.where(weather_all['timestamp'].dt.month <= 8, 2,
                         np.where(weather_all['timestamp'].dt.month <= 11, 3, 4))))

In [ ]:
# air_temp

print('NaN before impute: {}'.format(weather_all.air_temperature.isnull().sum()))

air_temp_filler = weather_all.groupby(['site_id', 'year','month','hour'])['air_temperature'].median()

for site_id, year, month, hour in air_temp_filler.index:
    weather_all.loc[(weather_all['site_id'] == site_id) & 
                    (weather_all['year'] == year) & 
                    (weather_all['month'] == month) & 
                    (weather_all['hour'] == hour) &
                    (weather_all.air_temperature.isnull()), 'air_temperature']\
    = air_temp_filler[(site_id, year, month, hour)]

print('NaN after impute: {}'.format(weather_all.air_temperature.isnull().sum()))

In [ ]:
weather_all.groupby(['site_id', 'year','month'])['air_temperature'].describe().head(12)

In [ ]:
# cloud_average

print(weather_all.cloud_coverage.isnull().sum(), 
      'Pecentage: {:.2f}'.format(weather_all.cloud_coverage.isnull().sum() / weather_all.shape[0]))

In [ ]:
# study the cloud coverage (take a small portion of data and study its pattern)

fig, ax = plt.subplots(len(set(weather_all['site_id'].values)),1, figsize = (20, 60))
idx = 0
for site_id in set(weather_all['site_id'].values):
    ax[idx].plot(weather_all[(weather_all['site_id'] == site_id) &
                             (weather_all['year'] == 2016)][['timestamp', 'cloud_coverage']]\
    .set_index('timestamp'))
    ax[idx].set_title('Site ID: {}'.format(site_id), fontsize = 10)
    idx += 1
    
    

In [ ]:
# site 7 and 11 did not have cloud_coverage data at all. Hence inpute the rest NaN's with -1.
weather_all.groupby(['site_id','year','day'])['cloud_coverage'].mean()\
[weather_all.groupby(['site_id','year','day'])['cloud_coverage'].mean().isnull()].index.tolist()

In [ ]:
# from the analysis, the cloud coverage should be similar for the same day of each month.

cloud_coverage_fillter = weather_all.groupby(['site_id','year','day'])['cloud_coverage'].median()

print("NaN before impute: {}".format(weather_all.cloud_coverage.isnull().sum()))

for site_id, year, day in cloud_coverage_fillter.index:
    weather_all.loc[(weather_all['site_id'] == site_id) & 
               (weather_all['year'] == year) & 
               (weather_all['day'] == day) & 
                    (weather_all.cloud_coverage.isnull()), 'cloud_coverage'] = cloud_coverage_fillter[(site_id, year, day)]

weather_all['cloud_coverage'].fillna(-999, inplace = True)
print("NaN after impute: {}".format(weather_all.cloud_coverage.isnull().sum()))

In [ ]:
weather_all.groupby(['site_id', 'year','day'])['cloud_coverage'].describe().head(12)

In [ ]:
# Temperaure should be the same for the same hour of each day within a specific month. 
# Therefore we impute the missing values this way.
    
dew_temperature_filler = weather_all.groupby(['site_id', 'year', 'month', 'hour'])['dew_temperature'].median()

print('NaN before impute: {}'.format(weather_all.dew_temperature.isnull().sum()))

for site_id, year, month, hour in dew_temperature_filler.index:
    weather_all.loc[(weather_all['site_id'] == site_id) & 
                    (weather_all['year'] == year) & 
                    (weather_all['month'] == month) & 
                    (weather_all['hour'] == hour) &
                    (weather_all.dew_temperature.isnull()), 'dew_temperature']\
    = dew_temperature_filler[(site_id, year, month, hour)]

print('NaN after impute: {}'.format(weather_all.dew_temperature.isnull().sum()))

In [ ]:
weather_all.groupby(['site_id', 'year','month'])['dew_temperature'].describe().head(12)

In [ ]:
# precip_depth_1_hr

print('Nan count:', weather_all.precip_depth_1_hr.isnull().sum(), 'Percentage: {:.2f}'.format(weather_all.precip_depth_1_hr.isnull().sum() / weather_all.shape[0]))


In [ ]:
# site_id 5, 12, 1 did not have any percipitation records. Therefore impute -999.
weather_all.groupby(['site_id','year','month']).precip_depth_1_hr\
.apply(lambda x: x.isnull().sum() / (x.count() + x.isnull().sum())).sort_values(ascending = False)[:108].index.tolist()

In [ ]:
print('NaN before impute: {}'.format(weather_all.precip_depth_1_hr.isnull().sum()))

precipt_depth_filler = weather_all.groupby(['site_id','year','month']).precip_depth_1_hr.median()

for site_id, year, month in precipt_depth_filler.index:
    weather_all.loc[(weather_all['site_id'] == site_id) & 
                    (weather_all['year'] == year) & 
                    (weather_all['month'] == month) & 
                    (weather_all.precip_depth_1_hr.isnull()), 'precip_depth_1_hr']\
    = precipt_depth_filler[(site_id, year, month)]

weather_all.precip_depth_1_hr.fillna(-999, inplace = True)

print('NaN after impute: {}'.format(weather_all.precip_depth_1_hr.isnull().sum()))

In [ ]:
# sea_level_pressure

print('Nan count:', weather_all.sea_level_pressure.isnull().sum(), 
      'Percentage: {:.2f}'.format(weather_all.sea_level_pressure.isnull().sum() / weather_all.shape[0]))


In [ ]:
# site 5 did not have sea level pressure data. therefore impute with -1
weather_all.groupby(['site_id', 'year','month']).sea_level_pressure.median()\
[weather_all.groupby(['site_id', 'year','month']).sea_level_pressure.median().isnull()]

In [ ]:
print('NaN before impute: {}'.format(weather_all.sea_level_pressure.isnull().sum()))

sea_level_filler = weather_all.groupby(['site_id','year', 'month']).sea_level_pressure.median()

for site_id, year, month in sea_level_filler.index:
    weather_all.loc[(weather_all['site_id'] == site_id) & 
                    (weather_all['year'] == year) & 
                    (weather_all['month'] == month) & 
                    (weather_all.sea_level_pressure.isnull()), 'sea_level_pressure']\
    = sea_level_filler[(site_id, year, month)]

weather_all.sea_level_pressure.fillna(-999, inplace = True)

print('NaN after impute: {}'.format(weather_all.sea_level_pressure.isnull().sum()))

In [ ]:
# wind_direction

print('Nan count:', weather_all.wind_direction.isnull().sum(), 
      'Percentage: {:.2f}'.format(weather_all.wind_direction.isnull().sum() / weather_all.shape[0]))



In [ ]:
print('NaN before impute: {}'.format(weather_all.wind_direction.isnull().sum()))

wind_direction_filler = weather_all.groupby(['site_id','year','month','hour']).wind_direction.median()

for site_id, year, month, hour in wind_direction_filler.index:
    weather_all.loc[(weather_all['site_id'] == site_id) & 
                    (weather_all['year'] == year) & 
                    (weather_all['month'] == month) &
                    (weather_all['hour'] == hour) &
                    (weather_all.wind_direction.isnull()), 'wind_direction']\
    = wind_direction_filler[(site_id, year, month, hour)]

print('NaN after impute: {}'.format(weather_all.wind_direction.isnull().sum()))

In [ ]:
# wind_speed

print('NaN before impute: {}'.format(weather_all.wind_speed.isnull().sum()))

wind_speed_filler = weather_all.groupby(['site_id','year','month','hour']).wind_speed.median()

for site_id, year, month, hour in wind_speed_filler.index:
    weather_all.loc[(weather_all['site_id'] == site_id) & 
                    (weather_all['year'] == year) & 
                    (weather_all['month'] == month) &
                    (weather_all['hour'] == hour) &
                    (weather_all.wind_speed.isnull()), 'wind_speed']\
    = wind_speed_filler[(site_id, year, month, hour)]

print('NaN after impute: {}'.format(weather_all.wind_speed.isnull().sum()))

In [ ]:
weather_all.describe()

In [ ]:
df_col_size(weather_all)

In [ ]:
weather_all['site_id'] = weather_all['site_id'].astype(np.int8)
weather_all['air_temperature'] = weather_all['air_temperature'].astype(np.float16)
weather_all['cloud_coverage'] = weather_all['cloud_coverage'].astype(np.float16)
weather_all['dew_temperature'] = weather_all['dew_temperature'].astype(np.float16)
weather_all['precip_depth_1_hr'] = weather_all['precip_depth_1_hr'].astype(np.float16)
weather_all['sea_level_pressure'] = weather_all['sea_level_pressure'].astype(np.float16)
weather_all['wind_direction'] = weather_all['wind_direction'].astype(np.float16)
weather_all['wind_speed'] = weather_all['wind_speed'].astype(np.float16)
weather_all['year'] = weather_all['year'].astype(np.float16)
weather_all['month'] = weather_all['month'].astype(np.float16)
weather_all['day'] = weather_all['day'].astype(np.float16)
weather_all['week'] = weather_all['week'].astype(np.float16)
weather_all['weekday'] = weather_all['weekday'].astype(np.float16)
weather_all['hour'] = weather_all['hour'].astype(np.float16)
weather_all['season'] = weather_all['season'].astype(np.float16)

df_col_size(weather_all)

#### 1.2.3 Train/Test Dataset

convert data type to prepare data merge.

In [ ]:
df_col_size(train)
train.head()

In [ ]:
train['building_id'] = train['building_id'].astype(np.int16)
train['meter'] = train['meter'].astype(np.int8)
train['timestamp'] = pd.to_datetime(train['timestamp'])

df_col_size(train)

In [ ]:
df_col_size(test)
test.head()

In [ ]:
test['building_id'] = test['building_id'].astype(np.int16)
test['meter'] = test['meter'].astype(np.int8)
test['timestamp'] = pd.to_datetime(test['timestamp'])

df_col_size(test)

In [ ]:
# Merge dataframes to prepare training dataset

print(train.building_id.nunique(), building_meta.building_id.nunique())

In [ ]:
train_df = train.merge(building_meta, how = 'inner', on = 'building_id' )

df_col_size(train_df)
train_df.head()


In [ ]:
train_df = train_df.merge(weather_all, how = 'left', on = ['site_id', 'timestamp'])

df_col_size(train_df)
train_df.head()

del train

In [ ]:
train_df.to_csv('../working/train_df.csv', index = False)

In [ ]:
gc.collect()

In [ ]:
train_df.columns

In [ ]:
id_col = ['site_id','building_id','meter']
train_col = ['primary_use','square_feet','year_built','floor_count',
            'air_temperature','cloud_coverage','dew_temperature',
            'precip_depth_1_hr','sea_level_pressure','wind_direction',
            'wind_speed','month','day','week','weekday','hour','season']
target_col = ['meter_reading']

train_df = pd.read_csv('', compression = 'zip')

## 2. Baseline Model

#### 2.1 GradiantBoostingRegressor

In [ ]:
from sklearn.ensemble import GradiantBoostingRegressor

train_set = train_df[train_df.timestamp]

In [ ]:
from catboost import Pool, CatBoostRegressor
import math

class RMSLE():
    
    def get_final_error(self, error, weight):
        return error / (weight + 1e-38)
    
    def is_max_optimal(self):
        return False
    
    def evaluate(self, approxes, target, weight):
        '''
            approxes is list of indexed containers
            (containers with only __len__ and __getitem__ defined), one container
            per approx dimension. Each container contains floats.
            weight is one dimensional indexed container.
            target is float.
            weight parameter can be None.
            Returns pair (error, weights sum)
            
        '''
        
        assert len(approxes) == 1
        assert len(target) == len(approxes[0])
        
        approx = approxes[0]
        
        error_sum = 0.0
        weight_sum = 0.0
        
        for i in range(len(approx)):
            w = 1.0 if weight is None else weight[i]
            weight_sum += w
            error_sum += ((math.log(approx[i] + 1) - math.log(target[i] + 1))**2) * w

        return math.sqrt(error_sum / weight_sum), weight_sum        

In [ ]:
demo = train_df[(train_df.site_id == 1) &
        (train_df.building_id == 119) &
        (train_df.meter == 0)]

train_pool = Pool(data = demo.loc[demo.timestamp.dt.month <= 9, train_col],
                 label = demo.loc[demo.timestamp.dt.month <= 9, target_col],
                 cat_features = ['primary_use'])

validation_pool = Pool(data = demo.loc[demo.timestamp.dt.month > 9, train_col],
                 label = demo.loc[demo.timestamp.dt.month > 9, target_col],
                 cat_features = ['primary_use'])

In [ ]:
model = CatBoostRegressor(iterations = 1000, learning_rate = 0.1, depth = 6,
                         loss_function = 'RMSE',
                         eval_metric = RMSLE(),
                         random_seed = 1234,
                         use_best_model = True)

model.fit(train_pool, eval_set = validation_pool, silent = True, early_stopping_rounds = 50)


In [ ]:
DataFrame({'names': model.feature_names_,
           'importance': model.feature_importances_
          }).sort_values(by = 'importance', ascending = False)
           
       

In [ ]:
model.drop_unused_features()
model.feature_names_

In [0]:
import pandas as pd
building_metadata = pd.read_csv("../input/ashrae-energy-prediction/building_metadata.csv")
sample_submission = pd.read_csv("../input/ashrae-energy-prediction/sample_submission.csv")
test = pd.read_csv("../input/ashrae-energy-prediction/test.csv")
train = pd.read_csv("../input/ashrae-energy-prediction/train.csv")
weather_test = pd.read_csv("../input/ashrae-energy-prediction/weather_test.csv")
weather_train = pd.read_csv("../input/ashrae-energy-prediction/weather_train.csv")